In [47]:
import ujson
import pprint
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from operator import itemgetter
import multiprocessing
from copy import deepcopy

from itertools import ifilter


pp = pprint.PrettyPrinter(indent=4)
%pylab inline

Populating the interactive namespace from numpy and matplotlib
time: 306 ms


/usr/local/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [48]:
# Testing auto adding time output to all cells
'''
#To install...
wget https://raw.githubusercontent.com/cpcloud/ipython-autotime/master/autotime.py
#Make available via
jupyter nbextension install /Users/patrickmulrooney/class/notebooks/pjmulroo/DSECapstone/autotime.py
'''
 
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 7.59 ms


In [49]:
def log_progress(sequence, every=None, size=None):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    label.value = u'{index} / {size}'.format(
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = str(index or '?')

# for _line in log_progress(lines,every=1):
#     _line = _line.strip()

time: 136 ms


In [50]:
!ls /Users/patrickmulrooney/Downloads/endomondo/

endomondo_parsedaa.json endomondo_parsedae.json endomondo_parsedai.json
endomondo_parsedab.json endomondo_parsedaf.json endomondo_parsedaj.json
endomondo_parsedac.json endomondo_parsedag.json
endomondo_parsedad.json endomondo_parsedah.json
time: 254 ms


# Overview

-- Data pulled from 'http://jmcauley.ucsd.edu/data/endomondo/endomondo.json.gz' and 'http://jmcauley.ucsd.edu/data/endomondo/endomondoHR.json.gz' on 2016.11.26.

-- MD5 of uncompressed files at time or download:

```
753c9b9afdb9e27076b831985c52bf33  endomondo.json
800036c402949822a71cdf69143c56d3  endomondoHR.json
```


# Parse endomondo.json

Following commands used to get data in proper format:


```
cd ~/Downloads/
# Replace all the single quotes with double, valid json
perl -pe "s/\'/\"/g" endomondo.json > endomondo_parsed1.json
# Add comma to end of each line, make valid json
perl -pe "s/$/,/" endomondo_parsed1.json > endomondo_parsed2.json 
# Remove temp file
rm endomondo_parsed1.json
# Split file into more managable files
mkdir endomondo
cd endomondo
split -l 100000 ../endomondo_parsed2.json endomondo_parsed
# Append [ to beginning of file, and replace last ',' with closing bracket
# to make files valid JSON
for i in `ls`; do sed "1s/^/\[/;$ s/.$/]/" $i > $i.json;done
```

In [51]:
# 8hr 1min
import psycopg2
from collections import defaultdict

dbname = 'endomondo'
conn = psycopg2.connect("dbname=endomondo user=patrickmulrooney")

# Open a cursor to perform database operations
cur = conn.cursor()

endoFiles = ['a','b','c','d','e','f','g','h','i','j']

problem_queries = []
problem_queries2 = []

for _file in log_progress(endoFiles, every=1):
    
    with open("/Users/patrickmulrooney/Downloads/endomondo/endomondo_parseda%s.json"%_file, 'r') as f:
        # Just testing the import, runs out of memory
        # TODO: Figure out why or how to deal with.
        endo = ujson.load(f)
        #endo.append(ujson.load(f))

        
    print "Got %s records"%len(endo)

    # time: 41min 15s
    # Version3 of cell above, inplace instead of new, using del instead of pop.
    # Go through each record and find out which keys it contains whose values are a list.
    # All list values should be part of a time series which we should combine into a single key value.
    # Not sure why it is so slow, should be faster.

    endoListTypes = ['timestamp', 'altitude', 'latitude', 'longitude', 'speed']

    seriesLength = []

    #for _endo in log_progress(endo, every=5):
    # Deal with stopped translation.
    for _i, _endo in enumerate(endo):

        if _endo['sport'] == "bike (transport)":
            _endo['sport'] = "bike_transport"
        else:
            _endo['sport'] = _endo['sport'].replace(" ", "_")
            _endo['sport'] = _endo['sport'].replace("-", "_")
        
        # Combine the list types for each record into one.
        # Probably should do more sanity checking of length of lists
        # If data not timestamped pretty useless
        _endo['series'] = []
        
        _endo['timestamp_included'] = True
        _endo['heart_rate_included'] = True
        _endo['altitude_included'] = True
        _endo['latitude_included'] = True
        _endo['longitude_included'] = True
        _endo['speed_included'] = True

        
        for j in xrange(len(_endo['timestamp'])):
            _timestamp = _endo['timestamp'][j]
               
            try:
                _heart_rate =_endo['heart_rate'][j]
            except:
                _endo['heart_rate_included'] = False
                _heart_rate = 'NULL'
            
            try:
                _altitude = _endo['altitude'][j]
            except:
                _endo['altitude_included'] = False
                _altitude = 'NULL'
                
            try:
                _latitude = _endo['latitude'][j]
            except:
                _endo['latitude_included'] = False
                _latitude = 'NULL'
                
            try:
                _longitude = _endo['longitude'][j]
            except:
                _endo['longitude_included'] = False
                _longitude = 'NULL'
                
            try:
                _speed =_endo['speed'][j]
            except:
                _endo['speed_included'] = False
                _speed = 'NULL'
                
            _endo['series'].append( [ _timestamp, _heart_rate, _altitude, _latitude, _longitude, _speed ] )

        # Remove unused keys
        if u'timestamp' in _endo.keys():
            del _endo['timestamp']
        if u'altitude' in _endo.keys():
            del _endo['altitude']
        if u'latitude' in _endo.keys():
            del _endo['latitude']
        if u'longitude' in _endo.keys():
            del _endo['longitude']
        if u'speed' in _endo.keys():
            del _endo['speed']
    

        _dict2 = {}
        _dict2["table"] = _endo['sport'] + "_by_workout"
        _dict2["gender"] = _endo['gender']
        _dict2["workoutId"] = _endo['id']
        _dict2["userId"] = _endo['userId']

        _dict2["start_time"] = _endo['series'][0][0]
        _dict2["duration"] = _endo['series'][-1][0] - _endo['series'][0][0]
        _dict2["start_altitude"] = _endo['series'][0][2]
        _dict2["start_latitude"] = _endo['series'][0][3]
        _dict2["start_longitude"] = _endo['series'][0][4]
        _dict2["series_length"] = len(_endo['series'])

        query2 = "INSERT INTO {} (gender, workoutid, userid, start_time, duration, start_altitude, start_latitude, start_longitude, series_length ) values ('{}', {}, {}, {}, {}, {}, {}, {}, {});".format(_dict2['table'], _dict2['gender'], _dict2['workoutId'], _dict2['userId'], _dict2['start_time'], _dict2['duration'], _dict2['start_altitude'], _dict2['start_latitude'], _dict2['start_longitude'], _dict2['series_length'])  
        try:
            cur.execute(query2)
        except:
            problem_queries.append(query2)

        _dict = defaultdict(lambda: 'NULL', {})

        _dict["table"] = _endo['sport']
        _dict["workoutId"] = _endo['id']

        for _series in _endo['series']:
            _dict["time"] = _series[0]

            if _endo["heart_rate_included"]:
                _dict["heart_rate"] = _series[1]

            if _endo["altitude_included"]:
                _dict["altitude"] = _series[2]

            if _endo["latitude_included"]:
                _dict["latitude"] = _series[3]

            if _endo['longitude_included']:
                _dict["longitude"] = _series[4]

            if _endo['speed_included']:
                _dict["speed"] = _series[5]


            query = "INSERT INTO {} (time, altitude, heart_rate, latitude, longitude, speed, workoutid) values ({}, {}, {}, {}, {}, {}, {});".format(_dict['table'], _dict['time'], _dict['altitude'], _dict['heart_rate'], _dict['latitude'], _dict['longitude'],_dict['speed'], _dict['workoutId'])
            try:
                cur.execute(query)
            except:
                problem_queries2.append(query)
                
        conn.commit()
        

Got 100000 records
Got 100000 records
Got 100000 records
Got 100000 records
Got 100000 records
Got 100000 records
Got 100000 records
Got 100000 records
Got 100000 records
Got 62190 records
time: 14h 10min 41s


In [ ]:
# ## Figure out later
# # # Debugging version, time: 29min 24s
# endoFiles = glob.glob("/Users/patrickmulrooney/Downloads/endomondo/*.json")

# endo = []

# for _file in log_progress(endoFiles, every=1):

#     with open(_file, 'r') as f:
#         # Just testing the import, runs out of memory
#         # TODO: Figure out why or how to deal with.
#         endo += ujson.load(f)
#         #endo.append(ujson.load(f))


In [ ]:
# print "Got %s records"%len(endo)

### Looks like only two variations of the data exist as far as series. Those that include the speed and those that don't.

```
{   (u'altitude', u'longitude', u'timestamp', u'latitude'): 203359,
    (u'altitude', u'longitude', u'timestamp', u'latitude', u'speed'): 49659}
```

In [ ]:
# problemEndo = []

In [ ]:
# # time: 41min 15s
# # Version3 of cell above, inplace instead of new, using del instead of pop.
# # Go through each record and find out which keys it contains whose values are a list.
# # All list values should be part of a time series which we should combine into a single key value.
# # Not sure why it is so slow, should be faster.

# endoListTypes = ['timestamp', 'altitude', 'latitude', 'longitude', 'speed']

# seriesLength = []

# #for _endo in log_progress(endo, every=5):
# # Deal with stopped translation.
# for _i, _endo in enumerate(endo):
#     if 'timestamp' not in _endo.keys():
#         continue
        
#     if not (set(endoListTypes) < set(_endo.keys())):
#         problemEndo.append(deepcopy(_endo))
#         endo[_i] = None
#         continue


#     # For debugging purposes, get a record of length of series for histogram
#     _seriesLen = len(_endo['timestamp'])
#     seriesLength.append(_seriesLen)

#     # Combine the list types for each record into one.
#     # Probably should do more sanity checking of length of lists
#     # If data not timestamped pretty useless
#     _endo['series'] = []
#     for j in xrange(_seriesLen):
#         # Speed is the only field missing, so deal with it
#         try:
#             _endo['speed_included'] = True
#             _endo['series'].append( [ _endo['timestamp'][j], _endo['altitude'][j], \
#                                            _endo['latitude'][j], _endo['longitude'][j], _endo['speed'][j] ] )

#         except:
#             _endo['speed_included'] = False
#             _endo['series'].append( [ _endo['timestamp'][j], _endo['altitude'][j], \
#                                            _endo['latitude'][j], _endo['longitude'][j] ] )

#     # Remove unused keys
#     del _endo['timestamp']
#     del _endo['altitude']
#     del _endo['latitude']
#     del _endo['longitude']
#     if u'speed' in _endo.keys():
#         del _endo['speed']


In [ ]:
# print len(problemEndo)
# print len(endo)

In [ ]:
# endo = list(ifilter(lambda x: x is not None, endo))

## Write parsed data to avoid needing to run this multiple times

In [ ]:
# # time: 3min 41s

# with open('/Users/patrickmulrooney/Downloads/endomondo/endoParsed.json', 'w') as f:
#      ujson.dump(endo, f)

# # Address later
# with open('/Users/patrickmulrooney/Downloads/endomondo/endoParsed_problem.json', 'w') as f:
#      ujson.dump(problemEndo, f)

## Address problem queries

### start with those into 'by_workout' tables

In [52]:
print len(problem_queries)
print len(problem_queries2)

15
5033
time: 5.82 ms


In [54]:
# for _i in problem_queries:
#     print _i

o = open("/Users/patrickmulrooney/Desktop/problems2.txt", 'w')

for _i in problem_queries2:
    o.write(_i + "\n")
    
o.close()

time: 14.7 ms


### Only real problem, others probably timeouts

```
INSERT INTO indoor_cycling (time, altitude, heart_rate, latitude, longitude, speed, workoutid) values (1343936052, 1e+25, 117, 0.236096, 32.623199, NULL, 77513570);
```

In [46]:
for _i in problem_queries2[:10]:
    print _i

INSERT INTO bike (time, altitude, heart_rate, latitude, longitude, speed, workoutid) values (1317918764, 6.6, NULL, 14.6544801258, 121.013972629, 16.38758, 36615563);
INSERT INTO bike (time, altitude, heart_rate, latitude, longitude, speed, workoutid) values (1317918804, 10.6, NULL, 14.6554908156, 121.013297634, 15.132809, 36615563);
INSERT INTO bike (time, altitude, heart_rate, latitude, longitude, speed, workoutid) values (1317918869, 14.2, NULL, 14.6554200724, 121.011925433, 5.188465, 36615563);
INSERT INTO bike (time, altitude, heart_rate, latitude, longitude, speed, workoutid) values (1317918965, 14.6, NULL, 14.6573976986, 121.011953345, 36.533875, 36615563);
INSERT INTO bike (time, altitude, heart_rate, latitude, longitude, speed, workoutid) values (1317919010, 12.6, NULL, 14.6575012989, 121.017422285, 52.348995, 36615563);
INSERT INTO bike (time, altitude, heart_rate, latitude, longitude, speed, workoutid) values (1317919053, 17.2, NULL, 14.6575150453, 121.020452175, 17.126165, 

## Move on to notebook 2
Link to [ExploreData - 2](ExploreData - 2.ipynb)

## Turns out all HR data is in the parent JSON as well. Going to parse and import everything, and remove entries from DB as appropriate.

#### Below is attempts before I realized this.

In [ ]:
# ## Figure out later
# # # Debugging version, time: 29min 24s
# endoFiles = glob.glob("/Users/patrickmulrooney/Downloads/endomondo/endomondo_parseda*.json")


# endoFiles = ['a','b','c','d','e','f','g','h','i','j']

# for _file in log_progress(endoFiles, every=1):
    
#     with open("/Users/patrickmulrooney/Downloads/endomondo/endomondo_parseda%s.json"%_file, 'r') as f:
#         # Just testing the import, runs out of memory
#         # TODO: Figure out why or how to deal with.
#         endo = ujson.load(f)
#         #endo.append(ujson.load(f))

        
#     print "Got %s records"%len(endo)

#     problemEndo = []

#     # time: 41min 15s
#     # Version3 of cell above, inplace instead of new, using del instead of pop.
#     # Go through each record and find out which keys it contains whose values are a list.
#     # All list values should be part of a time series which we should combine into a single key value.
#     # Not sure why it is so slow, should be faster.

#     endoListTypes = ['timestamp', 'altitude', 'latitude', 'longitude', 'speed']

#     seriesLength = []

#     #for _endo in log_progress(endo, every=5):
#     # Deal with stopped translation.
#     for _i, _endo in enumerate(endo):
#         if 'timestamp' not in _endo.keys():
#             continue

#         if not (set(endoListTypes) < set(_endo.keys())):
#             problemEndo.append(deepcopy(_endo))
#             endo[_i] = None
#             continue


#         # For debugging purposes, get a record of length of series for histogram
#         _seriesLen = len(_endo['timestamp'])
#         seriesLength.append(_seriesLen)

#         # Combine the list types for each record into one.
#         # Probably should do more sanity checking of length of lists
#         # If data not timestamped pretty useless
#         _endo['series'] = []
#         for j in xrange(_seriesLen):
#             # Speed is the only field missing, so deal with it
#             try:
#                 _timestamp = _endo['timestamp'][j]
#             except:
#                 _timestamp = None
                
#             try:
#                 _altitude = _endo['altitude'][j]
#             except:
#                 _altitude = None
                
#             try:
#                 _latitude = _endo['latitude'][j]
#             except:
#                 _latitude = None
                
#             try:
#                 _longitude = _endo['longitude'][j]
#             except:
#                 _longitude = None
                
#             try:
#                 _endo['speed_included'] = True
#                 _speed =_endo['speed'][j]
#             except:
#                 _endo['speed_included'] = False
#                 _speed = None

#         # Remove unused keys
#         if u'timestamp' in _endo.keys():
#             del _endo['timestamp']
#         if u'altitude' in _endo.keys():
#             del _endo['altitude']
#         if u'latitude' in _endo.keys():
#             del _endo['latitude']
#         if u'longitude' in _endo.keys():
#             del _endo['longitude']
#         if u'speed' in _endo.keys():
#             del _endo['speed']

#     print len(problemEndo)
#     print len(endo)

#     endo = list(ifilter(lambda x: x is not None, endo))

#     # time: 3min 41s

#     with open('/Users/patrickmulrooney/Downloads/endomondo/endoParsed%s.json'%_file, 'w') as f:
#          ujson.dump(endo, f)

#     # Address later
#     with open('/Users/patrickmulrooney/Downloads/endomondo/endoParsed%s_problem.json'%_file, 'w') as f:
#          ujson.dump(problemEndo, f)

## Handle Problems

In [6]:
# #"endoParsede_problem.json"

# ## Figure out later
# # # Debugging version, time: 29min 24s
# #endoFiles = glob.glob("/Users/patrickmulrooney/Downloads/endomondo/endoParsed%s_problem.json")
# endo = []

# endoFiles = ['a','b','c','d','e','f','g','h','i','j']

# for _file in log_progress(endoFiles, every=1):
    
#     with open("/Users/patrickmulrooney/Downloads/endomondo/endoParsed%s_problem.json"%_file, 'r') as f:
#         # Just testing the import, runs out of memory
#         # TODO: Figure out why or how to deal with.
#         endo += ujson.load(f)
#         #endo.append(ujson.load(f))

        
#     print "Got %s records"%len(endo)

Got 54080 records
Got 110664 records
Got 169269 records
Got 227500 records
Got 279363 records
Got 336800 records
Got 394341 records
Got 452815 records
Got 511833 records
Got 546762 records
time: 10min


In [ ]:


# for _endo in log_progress(endo, every=1):

In [12]:
# endo[0].keys()

[u'url',
 u'gender',
 u'altitude',
 u'userId',
 u'longitude',
 u'heart_rate',
 u'timestamp',
 u'latitude',
 u'sport',
 u'id']

time: 4.19 ms


In [16]:
# endo_keys = []

# for _endo in log_progress(endo, every=1):
#     _keys = _endo.keys()
    
#     _keys.remove("url")
#     _keys.remove("gender")
#     _keys.remove("userId")
#     _keys.remove("sport")
#     _keys.remove("id")

    
#     endo_keys.append(_keys)

time: 21min 48s


In [27]:
# for _i, _endo in enumerate(endo):
#     _keys = _endo.keys()
    
#     _keys.remove("url")
#     _keys.remove("gender")
#     _keys.remove("userId")
#     _keys.remove("sport")
#     _keys.remove("id")
    
#     # We should already have all of these...
#     if str.join(",", sorted(_keys)) == 'altitude,heart_rate,latitude,longitude,timestamp':
#         endo[_i] = None

time: 1min 3s


In [18]:
# key_count = {}

# for _keys in log_progress(endo_keys, every=1):
#     _keys = str.join(",", sorted(_keys))
    
#     try:
#         key_count[_keys] += 1
#     except:
#         key_count[_keys] = 1

time: 21min 39s


In [26]:
# key_count

{u'altitude,heart_rate,latitude,longitude,timestamp': 203360,
 u'altitude,heart_rate,speed,timestamp': 2797,
 u'altitude,heart_rate,timestamp': 11060,
 u'altitude,latitude,longitude,timestamp': 48747,
 u'altitude,speed,timestamp': 1517,
 u'altitude,timestamp': 1702,
 u'heart_rate,latitude,longitude,speed,timestamp': 1158,
 u'heart_rate,latitude,longitude,timestamp': 81564,
 u'heart_rate,speed,timestamp': 5313,
 u'heart_rate,timestamp': 22911,
 u'latitude,longitude,speed,timestamp': 73147,
 u'latitude,longitude,timestamp': 64386,
 u'speed,timestamp': 24942,
 u'timestamp': 4158}

time: 7.65 ms
